In [38]:
#백박강조 비전 데이터 전처리 코드

# 라이브러리 연결

In [ ]:
!sudo apt install tesseract-ocr
!sudo apt install tesseract-ocr-kor
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 37 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (5,389 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pytesseract
import cv2
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import math
from google.colab.patches import cv2_imshow

import re
import csv
from operator import itemgetter

# 특문
import string
import collections as ct
# 한국어
import sys
from importlib import reload
reload(sys)

<module 'sys' (built-in)>

In [ ]:
image_arr = os.listdir('/content/drive/MyDrive/images')

# 전체 모듈
* [0] 이미지 선택 (selectImage)
* [1.1] 이미지 한 장 입력 (loadImage)
* [1.2] 이미지 여러장 입력 (loadImages)
* [2.1] 이미지 한 장 출력 (showImage)
* [2.2] 이미지 여러장 출력 (showImages)
* [2.3] 이지지 회전 (image rotation)
* [3] 그레이로 변환 (To Gray)
* [4] 엣지 검출 (Edge Dectection)
* [5] 히스토그램 평탄화 (Historam Equalization)
* [6] 라인 검출 (Line Detection)
* [7] Smoothing
* [8] Closing
* [9] Opening
* [10] Adaptive Thresolding
* [11] Thresolding by OTSU
* [12] sharpning
* [13] 윤곽선 검출
* [14] 꼭지점 검출
* [15] 꼭지점 순서 정렬
* [16] 명암 조정
* [17] 방향 조정
* [18] OCR 실행
* [19] 결과 확인

### [0-1] 전체 이미지 입력 (Select All Images)

In [ ]:
def loadAll():
    image_arr = os.listdir('/content/drive/MyDrive/images')
    # for img in image_arr:
    #     imgArray = cv2.imread('./images/'+img)
    return image_arr

### [0-2] 처리할 이미지 선택 (Select Image(s))

In [ ]:
def selectImage():
    array = map(str, input("처리할 이미지를 선택하세요").split())
    for i in array:
        print(i)
    print('를 선택하셨습니다')
    return array

### [1.1] 이미지 1장 로드 (Load a Image)

In [ ]:
def loadImage(a):
    img = cv2.imread('/content/drive/MyDrive/images/'+str(a))
    if img is None:
        print('Imgae load failed!')
    return img

### [1.2] 이미지 여러장 로드 (Load Images)

In [ ]:
def loadImages(array):
    for e in array:
        img = cv2.imread('./images/'+str(e)+'.jpg')
        if img is None:
            print('Imgae load failed!')
    return img

### [2.1] 이미지 2장 출력 (Show a Image)

In [ ]:
def showImage(img):
  cv2_imshow(img)
    # cv2_imshow(a)
    # cv2.waitKey()
    # cv2.destroyAllWindows()

### [2.2] 이미지 여러장 출력 (Show Images)

In [ ]:
def showImages(array):
    for e in array:
        cv2_imshow(e)

### [2.3] 이미지 회전 (Image Rotation)

In [ ]:
def image_rotation(src):
    img_arr=[]
    
    img90 = cv2.rotate(src, cv2.ROTATE_90_CLOCKWISE) # 시계방향으로 90도 회전
    img180 = cv2.rotate(src, cv2.ROTATE_180) # 180도 회전
    img270 = cv2.rotate(src, cv2.ROTATE_90_COUNTERCLOCKWISE) # 반시계방향으로 90도 회전 

    img_arr.append(src)
    img_arr.append(img90)
    img_arr.append(img180)
    img_arr.append(img270)
    return img_arr

### [3] 그레이로 변환 (To Gray)
* COLOR_BG2GRAY 사용

In [ ]:
def toGray(src):
    gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    return gray

### [4] 엣지 검출 (Edge Detection)
* canny 사용

In [ ]:
def canny(src, p1, p2):
    # p1: min val, p2 : min val
    edges = cv2.Canny(src,p1,p2)
    return edges

### [5] 히스토그램 평탄화 (Histogram Equalization)
* clahe 사용

In [ ]:
def clahe(src):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl = clahe.apply(src)
    return cl

### [6] 라인 검출 (Line Detection)
* hough 사용

In [ ]:
def hough(src, p):
    rotate_dict={}
    # p : 변환된 그래프에서 라인을 검출하기 위한 최소 교차 수
    lines = cv2.HoughLines(src,1,np.pi/180,p)
#     print("len :",len(lines), lines[0])

    for line in lines:
        rho,theta = line[0]
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))
        cv2.line(src,(x1,y1),(x2,y2),(255,0,0),2)
        # print(theta)
        if theta in rotate_dict.keys():
            tmp = rotate_dict[theta]
            tmp+=1
            rotate_dict[theta] = tmp
        else:
            rotate_dict[theta] =1
    return src, rotate_dict

### [7] Smoothing
* Blur 처리

In [ ]:
def blur(src):
    blur = cv2.bilateralFilter(src,5,40,40)
    return src

### [8] Closing
* closing 처리

In [ ]:
def closing(src, p1, p2):
    # Closing: 팽창 -> 침식
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (p1,p2))
    closing = cv2.morphologyEx(src, cv2.MORPH_OPEN, kernel)
    return closing

### [9] Opening
* opening 처리

In [ ]:
def opening(src,p1, p2):
# Opening: 침식 -> 팽창
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (p1,p2))
    opening = cv2.morphologyEx(src, cv2.MORPH_OPEN, kernel)
    return opening

### [10] Adaptive Thresholding

In [ ]:
def thresh_adaptive(src, b,c):
    th = cv2.adaptiveThreshold(src, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,b,c)
    return th

### [11] Threshoding  OTSU 

In [ ]:
def thresh_otsu(src):
    # p1 : min, p2 : max
    th, thresh = cv2.threshold(src,0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    return thresh

### [12] Sharpning

In [ ]:
def sharpning(src):
    sharpning = np.array([[-1, -1, -1, -1, -1],
                         [-1, 2, 2, 2, -1],
                         [-1, 2, 9, 2, -1],
                         [-1, 2, 2, 2, -1],
                         [-1, -1, -1, -1, -1]]) / 9.0

    dst = cv2.filter2D(src, -1, sharpning)
    return dst

### [13] 윤곽선 검출
* Find Contour

In [ ]:
def drawCont(contours):
    # 출력 영상 설정

    find_cont = True
    for pts in contours:
    # 너무 작은 객체는 제외
        if cv2.contourArea(pts) < 2200:  
            # 외관선 구성 영역 2000보다 작을 시 skip
            # find_cont = False
            continue
        # 외곽선 근사화
        approx = cv2.approxPolyDP(pts, cv2.arcLength(pts, True)*0.02, True) 
        # 모서리 점 좌표 리스트

        # 컨벡스가 아니면 제외
        if not cv2.isContourConvex(approx) or len(approx) != 4: 
           # 영역이 사각형이 아닐 시 skip
            find_cont = False
            continue
        # cv2.polylines(img, [approx], True, (0, 255, 0), 2, cv2.LINE_AA) #외곽선 그리기
        srcQuad = reorderPts(approx.reshape(4, 2).astype(np.float32))
        #print(srcQuad)
        return srcQuad
    

def cont(src):
    contours, hierarchy = cv2.findContours(src, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    # cv2.drawContours(src, contours, -1, (255,0,0),2)
    
    return drawCont(contours)


### [14] 꼭지점 검출
* getPerspectiveTransform

In [ ]:
def transform(src, srcQuad, dstQuad, dw, dh):
    # 좌표 4개를 잡아줌, 사각형의 꼭짓점
    pers = cv2.getPerspectiveTransform(srcQuad, dstQuad) #투시 변환 행렬
    dst = cv2.warpPerspective(src, pers, (dw, dh), flags=cv2.INTER_CUBIC) #투시 변환 함수
    #print(srcQuad)
    return dst

### [15] 꼭지점 순서 정렬

In [ ]:
def reorderPts(pts): # 꼭지점 순서 정렬
    idx = np.lexsort((pts[:, 1], pts[:, 0]))  # 칼럼0 -> 칼럼1 순으로 정렬한 인덱스를 반환
    pts = pts[idx]  # x좌표로 정렬

    if pts[0, 1] > pts[1, 1]:
        pts[[0, 1]] = pts[[1, 0]]

    if pts[2, 1] < pts[3, 1]:
        pts[[2, 3]] = pts[[3, 2]]

    return pts

### [16] 명암 조정

In [ ]:
def contrast(src, ): # 명암 조정
    src_contrast = cv2.normalize(src, None, 0, 255, cv2.NORM_MINMAX) # 히스토그램 스트레칭/밝기 균일화
    blur = cv2.GaussianBlur(src_contrast,(5,5),0) #백색 노이즈 제거
    src_contrast = cv2.adaptiveThreshold(src_contrast,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,55,15) #그림자 제거
    return src_contrast

### [17] 방향 조정

In [ ]:
def rotate(src, p):
    rows, cols, _ = src.shape
    p = 90-math.degrees(p)
    #print(p)
    M = cv2.getRotationMatrix2D(((cols -1) / 2.0, (rows-1) / 2.0), -p, 1)
    dst = cv2.warpAffine(src, M, (cols, rows))
    return dst

### [18] OCR 출력

In [ ]:
def doOCR(src):
    #print(pytesseract.image_to_string(src, lang='kor+eng'))
    text = pytesseract.image_to_string(src, lang='kor+eng')
    return text

### [17] 최종 이미지 저장

In [ ]:
def saveImage(img, num):
  cv2.imwrite('/content/drive/MyDrive/result/'+num, img)


### [18] 엑셀로 저장

In [ ]:
global imageList
imageList = 2

In [ ]:
def excelMake():
  xcel = openpyxl.Workbook()
  excel_ws = excel.active
  excel_ws.title = 'sheet'
  excel_ws.cell(row = 1, column=1).value='원본 이미지'
  excel_ws.cell(row = 1, column=2).value='변환 이미지'
  excel_ws.cell(row = 1, column=3).value='OCR 결과'
  # img = openpyxl.drawing.image.Image('./images/12.jpg')
  # img.width=30*10
  # img.height=10*23
  excel_ws.column_dimensions['A'].width=38
  excel_ws.column_dimensions['B'].width=38
  excel_ws.column_dimensions['C'].width=38
  for i in range(2, 37):
    excel_ws.row_dimensions[i].height=174

In [ ]:
def excelSave(src, dst, ocr):
    excel = openpyxl.load_workbook('./output.xlsx')
    excel_ws = excel.active
    excel_ws.title = 'sheet'

    # dst 파일 불러오기
    # imgPath = '/result/'+dst
    # img = openpyxl.drawing.image.Image(imgPath)

    # dst 파일 크기조정
    # img.width=30*10
    # img.height=10*23

    # 원본 이미지, 변환 이미지, OCR 결과 저장
    srcNum = 'A'+str(imageList)
    excel_ws.add_image(src, srcNum)
    dstNum = 'B'+str(imageList)
    excel_ws.add_image(dst, dstNum)
    ocrNum = 'C'+str(imageList)
    excel_ws[ocrNum] = ocr

    imageList+=1

    excel.save('./output.xlsx')

### [19] 전처리 모듈 - 라인

In [ ]:
def line(src, org):
  rotate_dict={}

  gray_img = toGray(org)
  clahe_img = clahe(gray_img)
  blur_img = blur(clahe_img)
  closing_img = closing(blur_img, 5, 5)
  thresh_adapative_img = thresh_adaptive(closing_img, 55, 40)
  sharp_img = sharpning(thresh_adapative_img)
  canny_img = canny(sharp_img, 100, 200)
  hough_img, rotate_dict = hough(canny_img, 150)
  reverse_rotate_dict = dict(map(reversed, rotate_dict.items()))
  angle = reverse_rotate_dict[max(reverse_rotate_dict.keys())]
  constant= cv2.copyMakeBorder(org,100,100,100,100,cv2.BORDER_CONSTANT,value=[0,0,0])
  rotate_img = rotate(constant, angle)
  rotate_gray_img = toGray(rotate_img)
  return rotate_gray_img

## 코드

In [39]:
def basePipe(img):
    dw, dh = 900, 500 #보통 명함의 가로 세로 비율 9:5
    srcQuad = np.array([[0, 0], [0, 0], [0, 0], [0, 0]], np.float32) #기존 좌표 순서
    dstQuad = np.array([[0, 0], [0, dh], [dw, dh], [dw, 0]], np.float32) #출력 좌표 순서
    dst = np.zeros((dh, dw), np.uint8) #투시 변환 함수 0

    # 소스 불러오기
    src = loadImage(img)
    
    # 이미지 방향 회전
    img_array = image_rotation(src)
    
    text_arr_tmp = {}
    img_arr_tmp ={}

    i = 0

    for img in img_array:
      # # 소스 출력하기
      # showImage(src)

      # 그레이 변환
      gray_img = toGray(img)
      
      # clahe 
      clahe_img = clahe(gray_img)
      
      # sharpening
      sharp_img = sharpning(clahe_img)
      
      # closing
      closing_img = closing(sharp_img,5 ,5)
      
      # thresolding
      thresh_img = thresh_otsu(closing_img)
      
      # canny
      canny_img = canny(thresh_img, 100, 200)
      # showImage(canny_img)
      # contour detection
      srcQuad = cont(canny_img)

      try:
          dst_img = transform(img, srcQuad, dstQuad, dw, dh)
          dst_g_img = toGray(dst_img)
          showImage(dst_g_img)
          dst_img = thresh_adaptive(dst_g_img, 55, 20)
      except :
          dst_g_img = line(canny_img, img)
          dst_img = thresh_adaptive(dst_g_img, 55, 20)
      # showImage(dst_img)
      # 그레이 변환
      # final_img = toGray(dst_img)
          
      # # OCR 실행
      result = doOCR(dst_img)
      print(result)

      text_arr_tmp[i] = result
      img_arr_tmp[i] = dst_g_img
      i += 1

      #showImage(dst_img)

    output = ''

    test = 0
    for j in range(len(text_arr_tmp)):
      # result_cnt = len(text_arr_tmp[bestPic]) \
      #   - sum(c.isupper() for c in text_arr_tmp[bestPic]) \
      #   - sum(c.islower() for c in text_arr_tmp[bestPic]) \
      #   - sum(c.isdigit() for c in text_arr_tmp[bestPic]) \
      #   - sum(c.isspace() for c in text_arr_tmp[bestPic]) \
      #   - sum(v for k, v in ct.Counter(text_arr_tmp[bestPic]).items() if k in string.punctuation)
      result_cnt = len(re.sub('[^가-힣]', '', text_arr_tmp[j]))

      if result_cnt >= test:
        test = result_cnt
        output = text_arr_tmp[j]
        result_img = img_arr_tmp[j]

    return output, result_img

In [40]:
def main():
    #excelMake()
    imgArray = loadAll()
    #print(imgArray)
    fields = ['파일이름', 'OCR 결과']
    text_arr=[]
    # 기본 파이프라인
    for i in imgArray:
        text, result_img= basePipe(i)
        text_arr.append({'OCR 결과':text, '파일이름':int(i.split('.jpg')[0])})
        saveImage(result_img, i)
    data = sorted(text_arr, key=itemgetter('파일이름'))
    result = []
    for d in data:
      result.append(d)
    print(result)
    with open('result.csv', 'w', newline='', encoding='utf-8-sig') as f:
      write = csv.DictWriter(f, fieldnames=fields)
      write.writeheader()
      write.writerows(result)

In [41]:
main()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# import re
# strr = '한글테스트asdfasdfas123123\n\n!!\n\n44  ##4'
# # result_cnt = len(unicode(strr)) \
# #   - sum(c.isupper() for c in strr) \
# #   - sum(c.islower() for c in strr) \
# #   - sum(c.isdigit() for c in strr) \
# #   - sum(c.isspace() for c in strr) \
# #   - sum(v for k, v in ct.Counter(strr).items() if k in string.punctuation)


# result_cnt = len(re.sub('[^가-힣]', '', str(strr)))

# # a = len(strr)
# # b = print(sum(c.isupper() for c in strr))
# # c= sum(c.islower() for c in strr)
# # d= sum(c.isdigit() for c in strr)
# # e= sum(c.isspace() for c in strr)
# # f= sum(v for k, v in ct.Counter(strr).items() if k in string.punctuation)

# # print("%s %s %s %s% s %s", a, b, c, d, e, f)
# print(result_cnt)

5
